# Joint Space Width Jupyter Notebook

Run the Scanco IPL Joint Space Width (JSW) analysis in Python.

In [1]:
# Imports
import os
import sys
import itk
import datetime
import numpy as np
import SimpleITK as sitk

from itkwidgets import compare, view

from ormir_xct.util.hildebrand_thickness import calc_structure_thickness_statistics


# Define a global variable for erosion and dilation
# These values are taken directly from the JSW IPL scripts
MISC2 = 27
CALC = MISC2 + 8

# Set the image path
filename = 'J0010803_JOINT.nii'
image_dir = os.path.dirname(os.getcwd())
image_dir = os.path.join(image_dir, 'tests')
image_dir = os.path.join(image_dir, 'data')
seg_path = os.path.join(image_dir, filename)

# Read the image using SimpleITK
seg_image = sitk.ReadImage(seg_path)

view(seg_image, rotate=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageSS3; pr…

## Step 1: Image Padding

Pad the binary joint image with black space (zeros) to ensure the outside space is greater than the inside (joint) space.

In [2]:
# Image padding only in the X and Y direction
pad_image = sitk.ConstantPad(seg_image, [MISC2, MISC2, 0], [MISC2, MISC2, 0], 0)

# Use a binary threshold filter to set the segmentation image's value to 60 instead of the default 127
# We will use the difference in binary image values when thresholding out the joint space in later steps
pad_image = sitk.BinaryThreshold(pad_image, 1, 127, 60, 0)

# Write out the padded image
output_path = os.path.dirname(seg_path)
sitk.WriteImage(pad_image, os.path.join(output_path, 'PAD_IMG.mha'))

## Step 2: Image Dilation

Next we dilate the binary image by the constant **MISC2** that is taken from the original IPL script. Here we use the SimpleITK Ball structural element for dilation which should be similar to the Euclidean metric used in IPL. Once dilation is complete, remove any islands and fill any holes in the dilated mask.

In [3]:
# Binary image dilation using a sphere structural element
dilated_image = sitk.BinaryDilate(pad_image, [MISC2, MISC2, MISC2], sitk.sitkBall, 0, 60)

# Run connected components, sort the components by size, then take only the largest component
connected_image = sitk.ConnectedComponent(dilated_image, True)

# Sort the components by size
relabel_image = sitk.RelabelComponent(connected_image)

# Take only the largest component, set it's value to 127
first_component = sitk.BinaryThreshold(relabel_image, 1, 1, 127, 0)
filled_holes = sitk.BinaryFillhole(first_component, True, 127)

sitk.WriteImage(filled_holes, os.path.join(output_path, 'DILATE.mha'))

compare(seg_image, filled_holes, link_cmap=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=True, description='cmap'), Checkbox(va…

## Step 3: Image Erosion

Next erode the image and set the image's value to 30. Then, add the erosion and dilation images together. Areas with overlap between images will have a value of 90, and the joint space will have a value of 30. We can then use a binary threshold to extract the joint space.

In [4]:
# Erode the image, set the eroded mask's value to 30
eroded_image = sitk.BinaryErode(filled_holes, [CALC, CALC, CALC], sitk.sitkBall, 0, 127)
eroded_image = sitk.BinaryThreshold(eroded_image, 127, 127, 30, 0)

# Add the eroded image (value = 30) and joint image (value = 60) together. 
# Then threshold out JS image (value = 30)
add_image = sitk.Add(eroded_image, pad_image)
add_image = sitk.BinaryThreshold(add_image, 30, 30, 127, 0)

connected_image = sitk.ConnectedComponent(add_image, False)
relabel_image = sitk.RelabelComponent(connected_image)
js_mask = sitk.BinaryThreshold(relabel_image, 1, 1, 1, 0)

sitk.WriteImage(eroded_image, os.path.join(output_path, 'ERODE.mha'))
sitk.WriteImage(js_mask, os.path.join(output_path, 'JS_MASK.mha'))

compare(add_image, eroded_image, link_cmap=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=True, description='cmap'), Checkbox(va…

## Step 4: Compute the JS Parameters

Now we will compute JS volume, JSW mean, standard deviation, minimum, maximum, and JS asymmetry (JSW.Max / JSW.Min). A Python implememntation of the sphere fitting algorithm from IPL is used to compute JSW parameters. Algorithm is described in detail here:

`T. Hildebrand, P. Rüegsegger. A new method for the model-independent assessment of thickness in three-dimensional images. Journal of Microscopy. 1997.`

Outputs are saved to a CSV for comparision with the IPL JS workflow.

<img src='notebookImages/sphereFitting.png' width="250" height="250">

In [5]:
# Distance transform + JSW parameters
mask = sitk.GetArrayFromImage(js_mask)

# Needs to be fixed for masks with JS minimum < 1 voxel
# For now, set the minimum JSW value to twice the voel size (0.1214)
mean_thickness, mean_thicknessSTD, min_thickness, \
    max_thickness, thickness_map = calc_structure_thickness_statistics(mask, 0.0607, 0.1214)

sitk_image = sitk.GetImageFromArray(thickness_map)
sitk_image.SetOrigin(js_mask.GetOrigin()) # Set the origin so we can overlay the DT on the joint image
sitk_image.SetSpacing(js_mask.GetSpacing())
sitk_image.SetDirection(js_mask.GetDirection())
sitk.WriteImage(sitk_image, os.path.join(output_path, 'PY_DT.mha'))

# Get the volume of the JS
shape_stats = sitk.LabelShapeStatisticsImageFilter()
shape_stats.ComputeOrientedBoundingBoxOn()
shape_stats.Execute(js_mask)

jsv = shape_stats.GetPhysicalSize(1)
pixel_count = shape_stats.GetNumberOfPixels(1)

jsw_header = np.array([['Filename', 'Process Date', \
                        'JSV (mm3)', 'JSW.Mean (mm)', 'JSW.Mean_STD (mm)', \
                        'JSW.Min (mm)', 'JSW.Max (mm)', 'JSW.AS']], dtype=object)

jsw_params = np.array([[filename, datetime.datetime.now(), \
                        jsv, mean_thickness, mean_thicknessSTD, min_thickness, \
                        max_thickness, min_thickness/max_thickness]], dtype=object)

csv_data = np.vstack([jsw_header, jsw_params])
completed_string = csv_data.astype(str)
completed_string[1:,:] = csv_data[1:,:].astype('S7')

js_output = os.path.join(output_path, 'JSW_OUTPUT.csv')
np.savetxt(js_output, completed_string.astype(str), delimiter=',', fmt='%s')

view(thickness_map, rotate=True, axes=True, gradient_opacity=0.9)


Viewer(axes=True, geometries=[], gradient_opacity=0.9, point_sets=[], rendered_image=<itk.itkImagePython.itkIm…